<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/sdxl_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture
!pip install -q invisible_watermark transformers accelerate safetensors diffusers controlnet_aux==0.0.7 xformers mediapy
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline, DiffusionPipeline, KDPM2AncestralDiscreteScheduler, StableDiffusionXLPipeline, AutoencoderKL
import gc
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display
import os
import random
model_base = "stabilityai/stable-diffusion-xl-base-1.0"
v_autoencoder = "madebyollin/sdxl-vae-fp16-fix" # fix vae for run in fp16 precision without generating NaNs

vae = AutoencoderKL.from_pretrained(v_autoencoder, torch_dtype=torch.float16)

pipe = StableDiffusionXLPipeline.from_pretrained(
    model_base,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

pipe.safety_checker = None

pipe.to("cuda")
model_refiner = "stabilityai/stable-diffusion-xl-refiner-1.0"

pipe_refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_refiner,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

#pipe_refiner.to("cuda")
pipe_refiner.enable_model_cpu_offload()
#(Optional) Change the scheduler
pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(
pipe.scheduler.config, use_karras_sigmas=False
)
#generator = torch.Generator().manual_seed(42)

In [ ]:
#@title ##**Create** { display-mode: "form" }
prompt =  "Ukrainian flag by Pascal Campion, Luis Royo" #@param {type:"string"}
prompt2 = ""


negative_prompt = ''#@param {type:"string"}
negative_prompt2 = ''

scale = 10 #@param {type:"integer"}
steps = 30 #@param {type:"integer"}

height = 1024 #@param {type:"integer"}
width = 1024 #@param {type:"integer"}


from diffusers.schedulers.scheduling_dpmsolver_multistep import DPMSolverMultistepScheduler
from diffusers import StableDiffusionXLPipeline, StableDiffusionPipeline
from diffusers import DPMSolverMultistepScheduler, EulerDiscreteScheduler
from diffusers import KDPM2DiscreteScheduler, KDPM2AncestralDiscreteScheduler
from diffusers import EulerAncestralDiscreteScheduler, HeunDiscreteScheduler
from diffusers import LMSDiscreteScheduler

scheduler_name = "DPMPP_2M" #@param ["Euler_a","Euler", "Euler_K", "LMS", "LMS_Karras", "Heun", "DPMPP_2M", "DPMPP_2M_Karras", "DPMPP_2M_Lu", "DPMPP_2M_Stable", "DPMPP_2M_SDE", "DPMPP_2M_SDE_Karras", "DPMPP_2M_SDE_Lu", "DPMPP_2M_SDE_Stable", "DPM++_2M_Karras", "DPM_Fast"]

schedulers = {
    "Euler_K": (EulerDiscreteScheduler, {"use_karras_sigmas": True}),

    "DPMPP_2M": (DPMSolverMultistepScheduler, {}),
    "DPMPP_2M_Karras": (DPMSolverMultistepScheduler, {"use_karras_sigmas": True}),
    "DPMPP_2M_Lu": (DPMSolverMultistepScheduler, {"use_lu_lambdas": True}),
    "DPMPP_2M_Stable": (DPMSolverMultistepScheduler, {"euler_at_final": True}),

    "DPMPP_2M_SDE": (DPMSolverMultistepScheduler, {"algorithm_type": "sde-dpmsolver++"}),
    "DPMPP_2M_SDE_Karras": (DPMSolverMultistepScheduler, {"use_karras_sigmas": True, "algorithm_type": "sde-dpmsolver++"}),
    "DPMPP_2M_SDE_Lu": (DPMSolverMultistepScheduler, {"use_lu_lambdas": True, "algorithm_type": "sde-dpmsolver++"}),
    "DPMPP_2M_SDE_Stable": (DPMSolverMultistepScheduler, {"algorithm_type": "sde-dpmsolver++", "euler_at_final": True}),

    "DPM++_2M_Karras": (DPMSolverMultistepScheduler, {"use_karras_sigmas": True, "algorithm_type": "sde-dpmsolver++", "steps": 70}),
    "DPM_Fast": (DPMSolverMultistepScheduler, {"algorithm_type": "sde-dpmsolver++", "steps": 100}),

    "DPM2": (KDPM2DiscreteScheduler, {}),
    "DPM2_Karras": (KDPM2DiscreteScheduler, {"use_karras_sigmas": True}),
    "DPM2_a": (KDPM2AncestralDiscreteScheduler, {}),
    "DPM2_a_Karras": (KDPM2AncestralDiscreteScheduler, {"use_karras_sigmas": True}),

    "Euler": (EulerDiscreteScheduler, {}),
    "Euler_a": (EulerAncestralDiscreteScheduler, {}),

    "Heun": (HeunDiscreteScheduler, {}),

    "LMS": (LMSDiscreteScheduler, {}),
    "LMS_Karras": (LMSDiscreteScheduler, {"use_karras_sigmas": True}),
}

model_id = "stabilityai/stable-diffusion-xl-base-1.0"



# Установите параметры
params = {
    "prompt":prompt,
    "prompt_2":prompt2,
    "height":height,
    "width":width,
    "negative_prompt":negative_prompt,
    "negative_prompt_2":negative_prompt2,
    "guidance_scale":scale,
    "num_inference_steps":steps
}

save_dir = './samples_sdxl'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

seed = random.randint(0, 1000000000000000000)
sdxl_img.save(os.path.join(save_dir, f"seed_{seed}_steps_{params['num_inference_steps']}_{scheduler_name}.png"))
generator = torch.Generator(device='cuda').manual_seed(seed)

scheduler = DPMSolverMultistepScheduler.from_pretrained(
    model_id,
    subfolder="scheduler",
    **schedulers[scheduler_name][1],
)
pipe.scheduler = scheduler


sdxl_img = pipe(**params, generator=generator).images[0]

gc.collect()
torch.cuda.empty_cache()

In [ ]:
#@title ##**Show** { display-mode: "form" }
sdxl_img